In [0]:
 !pip install cptac

     |████████████████████████████████| 5.0MB 2.8MB/s 
     |████████████████████████████████| 112kB 61.4MB/s 
     |████████████████████████████████| 112kB 62.5MB/s 
  Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [0]:
import cptac
import pandas as pd

In [0]:
import statistics
import math

In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
def normalize(df):
  
  rightCol = False
  for col in df.columns:
    if col == "AAAS":
      rightCol = True
    else:
      rightCol = False
    values = df[col].tolist()
    sd = statistics.stdev(values)
    mean = statistics.mean(values)
    maxVal = 0
    minVal = math.inf
    highOutliers = []
    lowOutliers = []

    for i in range(len(values)):
      if values[i] >= 3*sd + mean:
        highOutliers.append(i)
      elif values[i] <= mean - 3*sd:
        lowOutliers.append(i)
      else:
        if values[i] > maxVal:
          maxVal = values[i]
        if values[i] < minVal:
          minVal = values[i]
  
    valRange = maxVal - minVal

    for i in range(len(values)):
      if i in highOutliers:
        values[i] = 2
      elif i in lowOutliers:
        values[i] = 1
      elif valRange > 0:
        values[i] = (values[i] -  minVal)/valRange + 1
      else:
        values[i] = 1

    df[col] = values

  return df         

In [0]:
def get_matched(data):
  patient_list = data.index.values.tolist()
  normal = []
  cancerous = []
  for ID in patient_list:
    if ID.endswith('N'):
      normal.append(ID[:-2])
    else:
      cancerous.append(ID)
  matched = [value for value in normal if value in cancerous]
  return matched

def get_ratio(gene, pairs, mrna, protein):
  cancer_ratio = []
  normal_ratio = []

  for patient in pairs:
    normal = patient + ".N"
    prot_cancer = protein.loc[patient][gene]
    prot_normal = protein.loc[normal][gene]
    mrna_cancer = mrna.loc[patient][gene]
    mrna_normal = mrna.loc[normal][gene]
    cancer_ratio.append(prot_cancer/mrna_cancer)
    normal_ratio.append(prot_normal/mrna_normal)

  return cancer_ratio, normal_ratio

def process_cancer(mrna, protein):
  mrna = normalize(mrna)
  protein = normalize(protein)

  matched_patients = [value for value in get_matched(protein) if value in get_matched(mrna)]
  
  gene_list = [value for value in list(protein.columns) if value in list(mrna.columns)] 


  control_data = {}
  cancer_data = {}
  for gene in gene_list: 
    cancer, control = get_ratio(gene, matched_patients, mrna, protein)
    control_data[gene] = control
    cancer_data[gene] = cancer

  cancer_ratios = pd.DataFrame(cancer_data, index=matched_patients)
  control_ratios = pd.DataFrame(control_data, index=matched_patients)

  return cancer_ratios, control_ratios

In [0]:
cptac.download(dataset='Endometrial')
en = cptac.Endometrial()
en_protein = en.get_proteomics()
en_mrna = en.get_transcriptomics()
en_cancer, en_control = process_cancer(en_mrna, en_protein)


entered
done with matched_patients
done with gene list


In [0]:
en_cancer.to_csv('en_cancer.csv')
en_control.to_csv('en_control.csv')
!mv en_cancer.csv drive/My\ Drive/
!mv en_control.csv drive/My\ Drive

In [0]:
for val in ['Luad', 'Ccrcc']:
  cptac.download(dataset=val)

In [0]:
lung = cptac.Luad()
cr = cptac.Ccrcc()

cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py, line 1)


In [0]:
lung_prot = lung.get_proteomics()


In [0]:
lung_prot.columns = lung_prot.columns.get_level_values(0)

In [0]:
lung_prot

In [0]:
lung_mrna = lung.get_transcriptomics()

In [0]:
new_lung_prot = {}

In [0]:
for colName in lung_prot.columns:
  if isinstance(lung_prot[colName], pd.DataFrame):
    continue
  col = lung_prot[colName].tolist()
  new_lung_prot[colName] = col

In [0]:
lung_prot = pd.DataFrame(new_lung_prot, index= lung_prot.index.values.tolist())

In [0]:
lung_cancer, lung_control = process_cancer(lung_mrna, lung_prot)

In [0]:
lung_cancer.to_csv("lung_cancer.csv")
lung_control.to_csv('lung_control.csv')
!mv lung_cancer.csv drive/My\ Drive/
!mv lung_control.csv drive/My\ Drive

In [0]:
cr_protein = cr.get_proteomics()
cr_mrna = cr.get_transcriptomics()
cr_protein.columns = cr_protein.columns.get_level_values(0)

new_cr_prot = {}
for colName in cr_protein.columns:
  if isinstance(cr_protein[colName], pd.DataFrame):
    continue
  col = cr_protein[colName].tolist()
  new_cr_prot[colName] = col

In [0]:
cr_protein = pd.DataFrame(new_cr_prot, index= cr_protein.index.values.tolist())
cr_cancer, cr_control = process_cancer(cr_mrna, cr_protein)

In [0]:
cr_cancer.head()

In [0]:
cr_control.head()

In [0]:
cr_cancer.to_csv("cr_cancer.csv")
#lung_control.to_csv('cr_control.csv')
cr_control.to_csv('cr_control.csv')
!mv cr_cancer.csv drive/My\ Drive/
!mv cr_control.csv drive/My\ Drive